# SMC SIMULATION K-12

In [1]:
import numpy as np
import pandas as pd
import orca
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import os; os.chdir('../../')
import warnings; warnings.simplefilter('ignore')

import pandana as pdna
from collections import OrderedDict
import pylogit as pl
from pylogit import nested_logit as nl
import math 
import pickle
import dill
import time
import random
import scipy.stats as st
from sklearn.neighbors import BallTree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim.utils import misc
import urbansim_templates

from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'WLCM_gen_tt'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [2]:
input_file_format = 'csv'
input_data_dir = '/home/data/spring_2019/base/'

formattable_fname_dict = {
    'parcels': 'parcels.{0}',
    'buildings': 'buildings.{0}',
    'jobs': 'jobs.{0}',
    'establishments': 'establishments.{0}',
    'households': 'households.{0}',
    'persons': 'persons.{0}',
    'rentals': 'craigslist.{0}',
    'units': 'units.{0}',
    'mtc_skims': 'mtc_skims.{0}',
    'beam_skims_raw': '30.skims-smart-23April2019-baseline.csv.gz',
    'beam_skims_imputed': 'beam_skims_imputed.{0}',
    # the following nodes and edges .csv's will be phased out and
    # replaced by travel model skims entirely
    'drive_nodes': 'drive_nodes.{0}',
    'drive_edges': 'drive_edges.{0}',
    'drive_access_vars': 'drive_net_vars.{0}',
    'walk_nodes': 'walk_nodes.{0}',
    'walk_edges': 'walk_edges.{0}',
    'walk_access_vars': 'walk_net_vars.{0}',
    'zones': 'zones.{0}',
    'zone_access_vars': 'zones_w_access_vars.{0}',
}

def format_fname_dict(formattable_fname_dict, format='csv'):
    formatted_dict = {
        k: v.format('csv')
        for k, v in formattable_fname_dict.items()}
    return formatted_dict

input_fnames = format_fname_dict(
            formattable_fname_dict, input_file_format)

In [3]:
orca.add_injectable('input_file_format', input_file_format)
orca.add_injectable('input_data_dir', input_data_dir)
orca.add_injectable('input_fnames', input_fnames)
orca.add_injectable('store', None)

In [4]:
orca.run(['initialize_network_small']) 
orca.run([ 'initialize_network_walk']) 
orca.run(['initialize_imputed_skims']) 
orca.run(['skims_aggregations']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s
Running step 'initialize_imputed_skims'
No imputed skims found. Creating them now.
Time to execute step 'initialize_imputed_skims': 85.62 s
Total time to execute iteration 1 with iteration value None: 85.62 s
Running step 'skims_aggregations'
Time to execute step 'skims_aggregations': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


In [5]:
orca.run(['SLCM_simulate','TOD_school_arrival_simulate','TOD_school_departure_simulate','TOD_school_distribution_simulate']) 

Running step 'SLCM_simulate'
Time to execute step 'SLCM_simulate': 886.15 s
Running step 'TOD_school_arrival_simulate'
Time to execute step 'TOD_school_arrival_simulate': 33.32 s
Running step 'TOD_school_departure_simulate'
Time to execute step 'TOD_school_departure_simulate': 11.72 s
Running step 'TOD_school_distribution_simulate'
Time to execute step 'TOD_school_distribution_simulate': 1.44 s
Total time to execute iteration 1 with iteration value None: 932.63 s


In [6]:
# In variables.py files

@orca.column('households', cache=True)
def hispanic_head_SMC(households):
    return households.hispanic_head.replace({'yes': 1, 'no': 0})

@orca.column('households', cache=True)
def tenure_SMC(households):
    return households.tenure.replace([1, 2],[1, 0])

@orca.column('households', cache=True)
def income_1(households):
    s = pd.cut(households.income, [-math.inf, 9999, 24999, 34999, 49999, 
                                        74999, 99999,149999, 199999, 249999, math.inf], 
                            labels = [1,2,3,4,5,6,7,8,9,10])
    return s 

@orca.column('households', cache=True)
def race_head_white(households):
    return (households.race_of_head == 1.0).astype(int)

@orca.column('households', cache=True)
def race_head_african_american(households):
    return (households.race_of_head == 2.0).astype(int)

@orca.column('households', column_name = 'race_head_indian/alaska', cache=True)
def race_head_alaska(households):
    return households.race_of_head.between(3.0, 5.0, inclusive=True).astype(int)
            
@orca.column('households', cache=True)
def race_head_asian(households):
    return (households.race_of_head == 6.0).astype(int)
            
@orca.column('households', column_name = 'race_head_hawaii/pacific', cache=True)
def race_head_hawaii(households):
    return (households.race_of_head == 7.0).astype(int)

@orca.column('households', column_name= 'race_head_other', cache=True)
def race_head_other(households):
    return (households.race_of_head == 8.0).astype(int)

@orca.column('households', column_name= 'race_head_2+races')
def race_head_two(households):
    return (households.race_of_head == 9.0).astype(int)

@orca.column('persons', cache=True)
def sex_SMC(persons):
    return persons.sex.replace([1, 2],[0, 1])

@orca.column('persons', cache=True)
def race_white(persons):
    return (persons.race_id == 1.0).astype(int)

@orca.column('persons', cache=True)
def race_african_american(persons):
    return (persons.race_id == 2.0).astype(int)

@orca.column('persons', column_name = 'race_indian/alaska', cache=True)
def race_alaska(persons):
    return persons.race_id.between(3.0, 5.0, inclusive=True).astype(int)
            
@orca.column('persons', cache=True)
def race_asian(persons):
    return (persons.race_id == 6.0).astype(int)
            
@orca.column('persons', column_name = 'race_hawaii/pacific', cache=True)
def race_hawaii(persons):
    return (persons.race_id == 7.0).astype(int)

@orca.column('persons', column_name= 'race_other', cache=True)
def race_other(persons):
    return (persons.race_id == 8.0).astype(int)

@orca.column('persons', column_name= 'race_2+races', cache=True)
def race_two(persons):
    return (persons.race_id == 9.0).astype(int)

In [7]:
# In utils.py file
@orca.injectable(cache = True)
def school_mode_choice_table():
    
    #Define bradcasting
    orca.broadcast(cast = 'household', onto = 'persons', cast_index = True, onto_on= 'household_id')
    orca.broadcast(cast = 'zones', onto = 'persons', cast_index= True, onto_on = 'zone_id_home')
    
    #Merging tables 
    df = orca.merge_tables(target = 'persons', tables = ['households', 'zones', 'persons'])
    
    df = pd.merge(df, orca.get_table('beam_skims_imputed').to_frame().reset_index(), how = 'left', 
              left_on = ['zone_id_home', 'zone_id_school'], 
              right_on = ['from_zone_id', 'to_zone_id'])
    
    list_var = ['sex_SMC', 'tenure_SMC', 'recent_mover', 'hispanic_head_SMC', 'income_1',
       'race_2+races', 'race_african_american', 'race_asian',
       'race_hawaii/pacific', 'race_indian/alaska', 'race_other', 'race_white',
       'race_head_2+races', 'race_head_african_american', 'race_head_asian',
       'race_head_hawaii/pacific', 'race_head_indian/alaska',
       'race_head_other', 'race_head_white', 'age', 'persons', 'cars',
       'workers', 'children', 'age_of_head', 'HS_ET',
       'sum_residential_units_gen_tt_WALK_TRANSIT_15',
       'total_jobs_gen_tt_WALK_TRANSIT_45',
       'avg_income_gen_tt_WALK_TRANSIT_30', 'sum_residential_units',
       'sum_persons_gen_tt_WALK_TRANSIT_15',
       'sum_income_gen_tt_WALK_TRANSIT_15', 'sum_income_gen_tt_CAR_45',
       'sum_persons_gen_tt_CAR_15', 'avg_income', 'total_jobs', 'sum_income',
       'sum_persons_gen_tt_CAR_45', 'sum_income_gen_tt_WALK_TRANSIT_45',
       'sum_residential_units_gen_tt_WALK_TRANSIT_45',
       'avg_income_gen_tt_CAR_30', 'sum_residential_units_gen_tt_CAR_15',
       'sum_persons_gen_tt_WALK_TRANSIT_45', 'total_jobs_gen_tt_CAR_45',
       'total_jobs_gen_tt_CAR_15', 'sum_persons',
       'sum_residential_units_gen_tt_CAR_45', 'sum_income_gen_tt_CAR_15',
       'total_jobs_gen_tt_WALK_TRANSIT_15', 'gen_tt_CAR',
       'gen_tt_DRIVE_TRANSIT', 'gen_cost_WALK_TRANSIT',
       'gen_tt_RIDE_HAIL_TRANSIT', 'gen_cost_RIDE_HAIL_TRANSIT',
       'gen_cost_CAR', 'gen_cost_DRIVE_TRANSIT', 'gen_cost_WALK', 'dist',
       'gen_cost_RIDE_HAIL', 'gen_tt_RIDE_HAIL', 'gen_cost_BIKE',
       'gen_tt_WALK', 'gen_cost_RIDE_HAIL_POOLED', 'gen_tt_WALK_TRANSIT',
       'gen_tt_BIKE', 'gen_tt_RIDE_HAIL_POOLED'] #List of variables in the same order as in model estimation
    
    students = df[(df.age> 4) & (df.age <=18) & (df.student == 1)][list_var]
    
    return students


In [8]:
## in models.py file
@orca.step()
def SMC_simulate(persons):
    """
      School Mode Choice simulation: Mode choice for school trips 
      1. Drive alone
      2. Shared
      3. Walk
      4. Bike
      5. Walk to transit
      6. Drive to transit
      7. School Bus
      
    """
    data = orca.get_injectable('school_mode_choice_table')
    
    #Running the simulation
    file_Name = "/home/juan/activitysynth/activitysynth/configs/SMC_ml.pkl"
    fileObject = open(file_Name,'rb')  
    mode_choice = pickle.load(fileObject)

    data = orca.get_injectable('school_mode_choice_table')
    choice_col = pd.Series(mode_choice.predict(data), index = data.index )
    
    #Adding column to persons table
    persons.update_col('school_mode_choice', choice_col)

In [9]:
orca.run(['SMC_simulate'])

Running step 'SMC_simulate'
Time to execute step 'SMC_simulate': 143.16 s
Total time to execute iteration 1 with iteration value None: 143.16 s
